# 멋진 작사가 만들기

### 1. 데이터 읽어오기

In [126]:
import os, re 
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [127]:
#glob로 모든 txt file 읽어오기
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

#읽어온 txt file raw_corpus리스트에 문장 단위로 저장
raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


In [128]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.

    if idx > 9: break   # 문장 10개 확인
        
    print(sentence)

Now I've heard there was a secret chord
That David played, and it pleased the Lord
But you don't really care for music, do you?
It goes like this
The fourth, the fifth
The minor fall, the major lift
The baffled king composing Hallelujah Hallelujah
Hallelujah
Hallelujah
Hallelujah Your faith was strong but you needed proof


### 2. 데이터 정제

In [129]:
#토큰화
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 소문자로 바꾸고, 양쪽 공백을 지웁니다
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 특수문자 양쪽에 공백을 넣고
    sentence = re.sub(r'[" "]+', " ", sentence) # 여러개의 공백은 하나의 공백으로 바꿉니다
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
    sentence = sentence.strip() # 다시 양쪽 공백을 지웁니다
    sentence = '<start> ' + sentence + ' <end>' # 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
    return sentence

In [130]:
#정제 데이터 구축하기
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue #길이 0
    if len(sentence.split()) >= 13: continue  #15개 이하(start,end포함)
    
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
    
    
corpus[:10] #정제결과 확인

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

### 3. 평가 데이터셋 분리

In [131]:
#tokenize() 함수로 데이터를 Tensor로 변환
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, #단어장의 크기:12,000 이상 
        filters=' ',
        oov_token="<unk>"
    )
    
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)  # corpus를 Tensor로 변환 
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   50    5 ...    0    0    0]
 [   2   17 3125 ...    0    0    0]
 [   2   34    7 ...    0    0    0]
 ...
 [   2  263  192 ...    0    0    0]
 [   2  127    5 ...    0    0    0]
 [   2    7   36 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f31b53f6280>


In [132]:
#단어사전 구축 인덱스 확인
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

#생성된 텐서를 소스와 타겟으로 분리해 모델 학습
src_input = tensor[:, :-1]   #소스 문장을 생성
tgt_input = tensor[:, 1:]   #타겟 문장을 생성

print(src_input[0])
print(tgt_input[0])

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to
[   2   50    5   91  304   62   57    9  974 6332    3    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0]
[  50    5   91  304   62   57    9  974 6332    3    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0]


In [133]:
#데이터셋 객체 생성
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1   

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 32), (256, 32)), types: (tf.int32, tf.int32)>

In [134]:
#총 데이터의 20% 를 평가 데이터셋
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2)

### 4. 인공지능 만들기

In [135]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256  # 워드 벡터의 차원수, 즉 단어가 추상적으로 표현되는 크기
hidden_size = 1024  #모델에 얼마나 많은 일꾼을 둘 것인가
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [136]:
# 데이터셋에서 데이터 한 배치만 불러오기
for src_sample, tgt_sample in dataset.take(1): break

model(src_sample)

<tf.Tensor: shape=(256, 32, 12001), dtype=float32, numpy=
array([[[ 3.7488193e-05,  3.1478823e-05, -2.1470565e-04, ...,
          1.6127748e-04, -4.9667480e-05,  2.3050488e-04],
        [ 2.1986695e-04, -1.1806750e-06, -5.5653433e-04, ...,
          1.3243180e-04, -7.2162838e-05,  4.6781183e-04],
        [ 2.4793742e-04,  4.9657177e-05, -5.3771725e-04, ...,
          1.8614915e-04,  1.0492243e-04,  4.5719004e-04],
        ...,
        [-8.5921347e-04,  2.0635976e-03, -3.0775103e-03, ...,
         -2.0881428e-03, -5.0196066e-03, -2.1982586e-04],
        [-8.3106931e-04,  2.0398549e-03, -3.0272522e-03, ...,
         -2.1556211e-03, -5.0455779e-03, -1.7801191e-04],
        [-8.0410868e-04,  2.0211078e-03, -2.9800639e-03, ...,
         -2.2161042e-03, -5.0670360e-03, -1.4026616e-04]],

       [[ 3.7488193e-05,  3.1478823e-05, -2.1470565e-04, ...,
          1.6127748e-04, -4.9667480e-05,  2.3050488e-04],
        [ 1.2725045e-04, -1.5368291e-04, -1.0009131e-04, ...,
          1.7742322e-04, 

In [137]:
model.summary()

Model: "text_generator_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      multiple                  3072256   
_________________________________________________________________
lstm_8 (LSTM)                multiple                  5246976   
_________________________________________________________________
lstm_9 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense_4 (Dense)              multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [139]:
#모델 학습하기
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
620/620 [==============================] - 484s 778ms/step - loss: 1.4346
Epoch 2/10
620/620 [==============================] - 424s 684ms/step - loss: 1.2281
Epoch 3/10
620/620 [==============================] - 235s 379ms/step - loss: 1.1173
Epoch 4/10
620/620 [==============================] - 236s 380ms/step - loss: 1.0222
Epoch 5/10
620/620 [==============================] - 235s 380ms/step - loss: 0.9362
Epoch 6/10
620/620 [==============================] - 236s 381ms/step - loss: 0.8568
Epoch 7/10
620/620 [==============================] - 236s 380ms/step - loss: 0.7832
Epoch 8/10
620/620 [==============================] - 236s 381ms/step - loss: 0.7161
Epoch 9/10
620/620 [==============================] - 236s 381ms/step - loss: 0.6567
Epoch 10/10
620/620 [==============================] - 237s 382ms/step - loss: 0.6060


epoch를 10번 실행시켜 loss가 0.6까지 떨어졌다.

In [140]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    while True:
        # 입력받은 문장의 텐서를 입력
        predict = model(test_tensor) 
        # 예측된 값 중 가장 높은 확률인 word index를 뽑기
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 예측된 word index를 문장 뒤에 붙임
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마침
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [141]:
#문장 생성 함수 실행
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , baby , so much dready got a job to do <end> '

---

## EXPLORATION 04 회고

여태까지 진행한 익스들은 프로젝트 전 실습을 하며 이해가 된 상태로 넘어가서, 프로젝트에 바로 응용할 수 있었는데 이번 익스는 연극 언어모델 실습을 하며 내가 이해를 하고 있는건지 처음 접하는거라 원래 이렇게 이해가 안가는건지 싶었다. 그래도 실습을 하며 필기한 내용 토대로 프로젝트를 진행하니 어떤 부분을 수정해야하는지, 어떤 부분이 헷갈렸던건지 이해 할 수 있었다. 무튼 자연어처리는 나에겐 매우 복잡하고 어려운 것이었는데 .. 프로젝트를 통해 또 한 단계 배울 수 있는 새로운 경험이었다.

-프로젝트를 진행하며 모호한 점  
enc_train, enc_val, dec_train, dec_val = <코드 작성> 부분에서 이전에 했던 실습을 참고하여 코드를 짰지만 계속 실패했다. 알고보니 가장 기본적이었던 import를 안해줬던 것.. 방법을 찾는데 매우 오래 걸렸다. 이 과정에서 random_state를 해줘야할지말지 고민하다가 결국 필수가 아니라는 생각에 적용하지 않았는데, random_state를 해야하는지? 하면 무슨 의미가 있는지 모호했다.

-프로젝트를 진행하며 아쉬웠던 점  
(1) 15개 이하로 만드는 과정에서 모든 데이터를 포괄하고 싶은데 함수를 몰라서 찾아봤다. 현재 적용한 split과 re.findall 두 가지가 가능했지만, re.findall은 반복하다보니 halleluya만 나오는 상황이 발생하여 sentence.split를 사용했다. 왜 한 문장만 반복되는지 궁금하기도 하며 아쉬웠다. 또한, 토큰화를 할 때 15개 이하로 만드는 과정을 포함하는 방법도 시도해보았는데 실패했다. 예상으로는 if를 무조건 사용하면서 else로 다른 걸 걸어줘야 하는 것 같은데.. 아무튼 나는 오류가 나서 아쉬웠다.  
(2) generate_text(lyricist, tokenizer, init_sentence="<start> i love", max_len=20)라서 처음엔 lyricist->model로 바꿔서 문자 생성에 성공했다. 그래서 모든 model->lyricist로 수정해서 실행했는데, i love까지만 문자 생성을 하고 실패했다. lyricist로 출력하지 못하고 최종적으로는 model로만 출력해 아쉬웠다.  

-프로젝트를 진행하며 궁금했던 점  
epoch가 처음엔 굉장히 빨리되었는데 계속 실행하다보니 매우 느려졌다. 마지막에는 확인해보니 620까지 올라갔다. 처음 할 땐 5분정도 걸렸는데 마지막엔 1시간 이상을 투자해야 했다. 실행할수록 epoch가 커지는 이유가 궁금했다. 이미 배운건데 기억을 못하는건가?아니면 당연한건가. 검색해봐도 검색식이 잘못되었는지 나오지 않았다. 처음 시도할 땐 epoch 30으로 0.789라는 결과를 만들었는데, 최종적으로는 epoch를 10으로 실행시켜도 오랜 시간이 걸렸기 때문에 10으로만 했다.

    
2022-01-20

---